# Traning with different size of signal

# prepare

## 1 - Load modules

In [ ]:
import sys
sys.path.append("..") # add self-defined module in the parent path
sys.path.append("../..") # add self-defined module in the parent path

from array import array
import datetime
import numpy as np
import pickle

from keras.models import Sequential, Model, load_model
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate, SimpleRNN, GRU, Masking, Lambda, Reshape, Dropout, RNN
from keras.optimizers import Adagrad, SGD, RMSprop, Adam
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Dense, Dropout, LeakyReLU, LSTM, Conv1D, SimpleRNN, Concatenate
from keras import optimizers
from keras.layers.normalization import BatchNormalization
import keras.backend as K

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc
from sklearn import preprocessing

import matplotlib as mpl
import matplotlib.pyplot as plt

from lfv_pdnn_code_v1.train.train_utils import *
from lfv_pdnn_code_v1.common.common_utils import *

selected_features = [0, 1, 2, 3, 5, 6, 7, 13, 14, 15, 16, 17, 18, 19]

# 2 - Load background samples

In [ ]:
# load background array
xb1 = np.load('/mnt/e/data/lfv/ntuples_last_run/TestData/data_npy/tree_bkg1.npy')
xb2 = np.load('/mnt/e/data/lfv/ntuples_last_run/TestData/data_npy/tree_bkg2.npy')
xb3 = np.load('/mnt/e/data/lfv/ntuples_last_run/TestData/data_npy/tree_bkg3.npy')
xb4 = np.load('/mnt/e/data/lfv/ntuples_last_run/TestData/data_npy/tree_bkg4.npy')
xb5 = np.load('/mnt/e/data/lfv/ntuples_last_run/TestData/data_npy/tree_bkg5.npy')
xb = np.concatenate((xb1, xb2, xb3, xb4, xb5))

data_path = "/mnt/e/data/new_ntuple/mc16a"
# load di_boson
directory = data_path + "/di_boson"
search_pattern = "*.npy"
absolute_file_list, file_name_list = get_file_list(directory, search_pattern)
xb_di_boson = np.array([])
for path in absolute_file_list:
    temp_array = np.load(path)
    if len(temp_array) == 0:
        continue
    if len(xb_di_boson) == 0:
        xb_di_boson = temp_array.copy()
    else:
        xb_di_boson = np.concatenate((xb_di_boson, temp_array))
print "xb_di_boson shape:", xb_di_boson.shape

# load top_quark
directory = data_path + "/top_quark"
search_pattern = "*.npy"
absolute_file_list, file_name_list = get_file_list(directory, search_pattern)
xb_top_quark = np.array([])
for path in absolute_file_list:
    temp_array = np.load(path)
    if len(temp_array) == 0:
        continue
    if len(xb_top_quark) == 0:
        xb_top_quark = temp_array.copy()
    else:
        xb_top_quark = np.concatenate((xb_top_quark, temp_array))
print "xb_top_quark shape:", xb_top_quark.shape

# load w_jets
directory = data_path + "/w_jets"
search_pattern = "*.npy"
absolute_file_list, file_name_list = get_file_list(directory, search_pattern)
xb_w_jets = np.array([])
for path in absolute_file_list:
    temp_array = np.load(path)
    if len(temp_array) == 0:
        continue
    if len(xb_w_jets) == 0:
        xb_w_jets = temp_array.copy()
    else:
        xb_w_jets = np.concatenate((xb_w_jets, temp_array))
print "xb_w_jets shape:", xb_w_jets.shape

# load z_ll
directory = data_path + "/z_ll"
search_pattern = "*.npy"
absolute_file_list, file_name_list = get_file_list(directory, search_pattern)
xb_z_ll = np.array([])
for path in absolute_file_list:
    temp_array = np.load(path)
    if len(temp_array) == 0:
        continue
    if len(xb_z_ll) == 0:
        xb_z_ll = temp_array.copy()
    else:
        xb_z_ll = np.concatenate((xb_z_ll, temp_array))
print "xb_z_ll shape:", xb_z_ll.shape

# Add all background together
xb = np.concatenate((xb_di_boson, xb_top_quark, xb_w_jets, xb_z_ll))
print "xb shape:", xb.shape

## 3 - Load selected signal samples
according to the previous results, use mass range (500 - 2400 GeV)

In [ ]:
mass_min = 5000
mass_max = 0
xs_studied = np.array([])
mass_scan_map = [5, 20]
data_path = "/mnt/e/data/new_ntuple/mc16a"
for i, mass in enumerate(mass_scan_map):
    # load signal
    xs_add = np.load(data_path + "/signal/emu/rpv_emu_{}00GeV.npy".format(mass))
    xs_temp = np.load(data_path + "/signal/etau/rpv_etau_{}00GeV.npy".format(mass))
    xs_add = np.concatenate((xs_add, xs_temp))
    xs_temp = np.load(data_path + "/signal/mutau/rpv_mutau_{}00GeV.npy".format(mass))
    xs_add = np.concatenate((xs_add, xs_temp))
    if len(xs_studied) == 0:
        xs_studied = xs_add.copy()
    else:
        xs_studied = np.concatenate((xs_studied, xs_add))
    xs_emu = modify_array(xs_add, weight_id = -1, 
                          select_channel = True, channel_id = -4,
                          norm = True, shuffle = True, shuffle_seed = 485)
    mass_sigma = get_mass_range(xs_emu[:, 0], xs_emu[:, -1])
    mass_min_temp = mass * 100 - mass_sigma
    mass_max_temp = mass * 100 + mass_sigma
    if mass_min_temp < mass_min:
        mass_min = mass_min_temp
    if mass_max_temp > mass_max:
        mass_max = mass_max_temp
        
# select emu channel and shuffle
xs_emu = modify_array(xs_studied, weight_id = -1, 
                      select_channel = True, channel_id = -4,
                      norm = True, shuffle = True)

# get bkg in range
xb_emu = modify_array(xb, weight_id = -1, remove_negative_weight = True, select_channel = True, channel_id = -4,
                      select_mass = True, mass_id = 0, mass_min = mass_min, mass_max = mass_max,
                      reset_mass = True, reset_mass_array = xs_emu, reset_mass_id = 0,
                      norm = True, shuffle = True)

# Make plots

## 1 - kinematic plots for emu channel

In [ ]:
MakePlots(xs_emu, xb_emu, 0, bins = 50, range = (0, 3000)  , density = True,
          xlabel = "di-Lepton mass / GeV", ylabel = "events", show_plot = True)
MakePlots(xs_emu, xb_emu, 1, bins = 50, range = (0, 1000)  , density = True,
          xlabel="Ele_pt / GeV"          , ylabel="Events"  , show_plot = True)
MakePlots(xs_emu, xb_emu, 2, bins = 50, range = (-3, 3)    , density = True,
          xlabel="Ele_eta"               , ylabel="Events"  , show_plot = True)
MakePlots(xs_emu, xb_emu, 3, bins = 50, range = (-3.2, 3.2), density = True,
          xlabel="Ele_phi"               , ylabel="Events"  , show_plot = True)
MakePlots(xs_emu, xb_emu, 5, bins = 50, range = (0, 5000)  , density = True,
          xlabel="Mu_pt / GeV"           , ylabel="Events"  , show_plot = True)
MakePlots(xs_emu, xb_emu, 6, bins = 50, range = (-3, 3)    , density = True,
          xlabel="Mu_eta"                , ylabel="Events"  , show_plot = True)
MakePlots(xs_emu, xb_emu, 7, bins = 50, range = (-3.2, 3.2), density = True,
          xlabel="Mu_phi"                , ylabel="Events"  , show_plot = True)

MakePlots(xs_emu, xb_emu, 13, bins = 50, range = (0, 3000)  , density = True,
          xlabel="MET_et"                , ylabel="Events"  , show_plot = True)
MakePlots(xs_emu, xb_emu, 14, bins = 50, range = (-3.2, 3.2), density = True,
          xlabel="MET_phi"               , ylabel="Events"  , show_plot = True)
MakePlots(xs_emu, xb_emu, 15, bins = 50, range = (0, 1000)  , density = True,
          xlabel="di-Lepton_pt"          , ylabel="Events"  , show_plot = True)
MakePlots(xs_emu, xb_emu, 16, bins = 50, range = (-3, 3)    , density = True,
          xlabel="di-Lepton_eta"         , ylabel="Events"  , show_plot = True)
MakePlots(xs_emu, xb_emu, 17, bins = 50, range = (-3.2, 3.2), density = True,
          xlabel="di-Lepton_phi"         , ylabel="Events"  , show_plot = True)
MakePlots(xs_emu, xb_emu, 18, bins = 50, range = (-3.2, 3.2), density = True,
          xlabel="di-Lepton_Dphi"        , ylabel="Events"  , show_plot = True)
MakePlots(xs_emu, xb_emu, 18, bins = 50, range = (0, 5)     , density = True,
          xlabel="di-Lepton_DR"          , ylabel="Events"  , show_plot = True)

# Training

## 1 - use 100% -> 1% signal for training
number of background events fixed

In [ ]:
#percentage_list = [0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]
percentage_list = [1]
#percentage_list = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.5, 1]
#percentage_list = [0.01, 0.1, 0.2, 0.5, 1]

# training with different percentage of signals
for percent in percentage_list:
    # set up training model
    node_num = 300
    model_deep = Sequential()
    model_deep.add(Dense(node_num, kernel_initializer='uniform', input_dim = len(selected_features)))
    model_deep.add(BatchNormalization())
    model_deep.add(Dense(node_num, kernel_initializer="glorot_normal", activation="relu"))
    model_deep.add(BatchNormalization())
    model_deep.add(Dense(node_num, kernel_initializer="glorot_normal", activation="relu"))
    model_deep.add(BatchNormalization())
    model_deep.add(Dense(node_num, kernel_initializer="glorot_normal", activation="relu"))
    model_deep.add(BatchNormalization())
    model_deep.add(Dense(node_num, kernel_initializer="glorot_normal", activation="relu"))
    model_deep.add(BatchNormalization())
    model_deep.add(Dense(node_num, kernel_initializer="glorot_normal", activation="relu"))
    model_deep.add(BatchNormalization())
    model_deep.add(Dense(1, kernel_initializer="glorot_uniform", activation="sigmoid"))
    # set loss, optimizer and evaluation metrics
    model_deep.compile(loss="binary_crossentropy", optimizer=SGD(lr=0.025, decay=1e-6), metrics=["accuracy"])

    # get part of signal events
    print "*" * 80
    print "use {}% signal for training".format(percent * 100)
    print "sig events quantity:", len(xs_emu) * percent
    print "bkg events quantity:", len(xb_emu)
    index_len = int(len(xs_emu) * percent)
    index = np.random.choice(xs_emu.shape[0], index_len, replace = False)
    xs_emu_part = xs_emu[index]
    #print "debug:", xs_emu_part[0:5]
    print "debug:", index[0:10]
    print "debug: ", xs_emu.shape[0], xs_emu_part.shape[0], index_len
    """
    xs_emu_part, x2, y1, y2 = train_test_split(xs_emu, np.ones(len(xs_emu)), 
                                               test_size= 1 - percent, 
                                               random_state=3456, shuffle=True)
    """
    xs_emu_part = modify_array(xs_emu_part, weight_id = -1, 
                               norm = True, shuffle = True)
    xb_emu_temp = modify_array(xb_emu, weight_id = -1, mass_id = 0,
                               reset_mass = True, reset_mass_array = xs_emu_part, reset_mass_id = 0,
                               norm = True, shuffle = True)
    
    # get training data
    x_emu_train, x_emu_test, y_emu_train, y_emu_test, xs_emu_test, xb_emu_test = \
    split_and_combine(xs_emu_part, xb_emu_temp, shuffle_before_return = True)
    # get test data
    #x1, x_emu_test, y1, y_emu_test, xs_emu_test, xb_emu_test = \
    #split_and_combine(xs_emu_part, xb_emu, shuffle_before_return = True)

    # select features wanted
    x_emu_train_selected = get_part_feature(x_emu_train, selected_features)
    x_emu_test_selected = get_part_feature(x_emu_test, selected_features)
    print "train data shape:", x_emu_train_selected.shape
    print "test data shape:", x_emu_test_selected.shape 

    # train the model
    train_history = model_deep.fit(x_emu_train_selected, y_emu_train, batch_size = 100, epochs = 30,
                    validation_split = 0.25 , sample_weight = x_emu_train[:, -1], verbose = 1)
    score = model_deep.evaluate(x_emu_test_selected, y_emu_test, verbose=0, sample_weight = x_emu_test[:, -1])
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

    # load test (cross validation) data
    xs_emu_test_selected = get_part_feature(xs_emu, selected_features) # use full sample for validation
    xb_emu_test_selected = get_part_feature(xb_emu, selected_features) #
    
    # display scores
    plt.hist(xs_emu[:, 0], bins = 100, weights = xs_emu[:, -1], range = (0, 10000), histtype='step', label='signal', density=True)
    plt.xlabel("mass")
    plt.ylabel("events")
    plt.show()
    PlotScores(xs_emu_test_selected, xb_emu_test_selected, model_deep, bins = 100, range = (0, 1), density = True)
    
    # summarize history for accuracy
    plt.plot(train_history.history['acc'])
    plt.plot(train_history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.ylim((0, 1))
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper center')
    plt.grid()
    plt.show()
    
    # summarize history for loss
    plt.plot(train_history.history['loss'])
    plt.plot(train_history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper center')
    plt.grid()
    plt.show()
    
    # make roc plots for signal
    print "roc for sig and bkg"
    plt.ylabel('tpr')
    plt.xlabel('fpr')
    predictions_dm = model_deep.predict(get_part_feature(x_emu_train, selected_features))
    fpr_dm, tpr_dm, threshold = roc_curve(y_emu_train, predictions_dm)
    plt.plot(fpr_dm, tpr_dm)
    predictions_dm = model_deep.predict(get_part_feature(x_emu_test, selected_features))
    fpr_dm_test, tpr_dm_test, threshold_test = roc_curve(y_emu_test, predictions_dm)
    plt.plot(fpr_dm_test, tpr_dm_test)
    plt.legend(['train', 'test'], loc='lower right')
    plt.grid()
    plt.show()
    print "auc for train:", auc(fpr_dm, tpr_dm)
    print "auc for test: ", auc(fpr_dm_test, tpr_dm_test)
    
    # clear model for next training
    K.clear_session()